# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2f301a0dc0>
├── 'a' --> tensor([[ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880]])
└── 'x' --> <FastTreeValue 0x7f2f301a0fd0>
    └── 'c' --> tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                        [-1.4732,  1.7046, -0.2569,  0.6856],
                        [ 0.4399, -1.3090,  1.2203,  0.5383]])

In [4]:
t.a

tensor([[ 1.7465,  1.4108, -0.5674],
        [ 0.1525,  0.6671, -1.4880]])

In [5]:
%timeit t.a

78.3 ns ± 0.566 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2f301a0dc0>
├── 'a' --> tensor([[-0.3370,  0.0364,  0.7381],
│                   [ 0.3024, -1.5486,  0.6221]])
└── 'x' --> <FastTreeValue 0x7f2f301a0fd0>
    └── 'c' --> tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                        [-1.4732,  1.7046, -0.2569,  0.6856],
                        [ 0.4399, -1.3090,  1.2203,  0.5383]])

In [7]:
%timeit t.a = new_value

78.4 ns ± 0.521 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880]]),
    x: Batch(
           c: tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                      [-1.4732,  1.7046, -0.2569,  0.6856],
                      [ 0.4399, -1.3090,  1.2203,  0.5383]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7465,  1.4108, -0.5674],
        [ 0.1525,  0.6671, -1.4880]])

In [11]:
%timeit b.a

71.4 ns ± 0.957 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5007,  0.0067, -1.7459],
               [-0.6597,  1.3801,  1.2005]]),
    x: Batch(
           c: tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                      [-1.4732,  1.7046, -0.2569,  0.6856],
                      [ 0.4399, -1.3090,  1.2203,  0.5383]]),
       ),
)

In [13]:
%timeit b.a = new_value

667 ns ± 9.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.16 µs ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.6 µs ± 253 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

273 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

261 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2e8bac16a0>
├── 'a' --> tensor([[[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]],
│           
│                   [[ 1.7465,  1.4108, -0.5674],
│                    [ 0.1525,  0.6671, -1.4880]]])
└── 'x' --> <FastTreeValue 0x7f2e8bac1d00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.8 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2e85599a00>
├── 'a' --> tensor([[ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880],
│                   [ 1.7465,  1.4108, -0.5674],
│                   [ 0.1525,  0.6671, -1.4880]])
└── 'x' --> <FastTreeValue 0x7f2e854b6fa0>
    └── 'c' --> tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                        [-1.4732,  1.7046, -0.2569,  0.6856],
                 

In [23]:
%timeit t_cat(trees)

43.7 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

87.9 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]],
       
               [[ 1.7465,  1.4108, -0.5674],
                [ 0.1525,  0.6671, -1.4880]]]),
    x: Batch(
           c: tensor([[[ 1.4667, -0.7995, -0.8379,  0.3497],
                       [-1.4732,  1.7046, -0.2569,  0.6856],
                       [ 0.4399, -1.3090,  1.2203,  0.5383]],
         

In [26]:
%timeit Batch.stack(batches)

109 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880],
               [ 1.7465,  1.4108, -0.5674],
               [ 0.1525,  0.6671, -1.4880]]),
    x: Batch(
           c: tensor([[ 1.4667, -0.7995, -0.8379,  0.3497],
                      [-1.4732,  1.7046, -0.2569,  0.6856],
                      [ 0.4399, -1.3090,  1.2203,  0.5383],
                      [ 1.4667, -0.7995, -0.8379,  0.3497],
                      [-1.4732,  

In [28]:
%timeit Batch.cat(batches)

201 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

581 µs ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
